In [1]:
import pandas as pd      
import os
import time
import glob
from pymongo import MongoClient
from pprint import pprint
from dotenv import load_dotenv
from binance.client import Client
from binance import AsyncClient
from datetime import datetime,timezone,timedelta
from pathlib import Path
import asyncio
from pytrends.request import TrendReq
pytrend = TrendReq()

In [2]:
# Load Environment Variables
load_dotenv()

# Gets MongoDB Connection String
MDB_CONNECTION_STRING = os.getenv('MDB_CONNECTION_STRING')

# Gets Binance Api Key and Api Secret
BINANCE_API_KEY = os.getenv('BINANCE_API_KEY')
BINANCE_API_SECRET = os.getenv('BINANCE_API_SECRET')

In [3]:
# Function to connect to the Mongo DB
def get_database():
    try:
        client = MongoClient(MDB_CONNECTION_STRING)
        db = client["project-02"]
        return db
    except Exception as e:
        print(e)

In [4]:
# Connect to the db
db = get_database()

In [5]:
# Test Connection
serverStatusResult=db.command("serverStatus")
print(serverStatusResult["version"])

5.0.6


In [6]:
# Gets the coinpairs to process from the database
def get_coinpairs():
    # Get the coinpairs from the Database
    db_coinpairs = db["coinpairs"].find({"exchange" : "binance"})

    # convert the dictionary objects to dataframe
    binance_coinpairs_df = pd.DataFrame(db_coinpairs)

    # see the magic
    coinpair_list = list(binance_coinpairs_df['pair'])
    return coinpair_list
coinpair_list = get_coinpairs()

In [7]:
# Function to get the timeframes stored in the database
def get_binance_timeframes_list(db):
    # Get the timeframes from the Database
    db_binance_timeframes = db["binance_timeframes"].find()

    # Convert the dictionary objects to dataframe
    binance_timeframes_df = pd.DataFrame(db_binance_timeframes)

    # Return the timeframes from the db
    return list(binance_timeframes_df['timeframe'])

binance_timeframes_list = get_binance_timeframes_list(db)
print(binance_timeframes_list)

['1m', '5m', '30m', '1h', '1d']


In [8]:
# Get latest inserted kline date for each Coin Pair
def get_pair_data(binance_timeframes_list, coinpair_list):
    complete_pair_tf = []
    from_timestamp = "3 months ago UTC"
    for timeframe in binance_timeframes_list:
        for pair in coinpair_list:
            # Get the coinpairs from the Database
            collection_name = pair+"_"+timeframe
            latest = db[collection_name].find().sort('open_time', -1 ).limit(1)
            # Exists, update collection by setting from_timestamp to lastest timestamp
            if collection_name in db.list_collection_names():
                complete_pair_tf.append([timeframe, pair, latest[0]["open_time"].strftime("%m/%d/%Y, %H:%M:%S"),collection_name, "is_update"])
            # Does not exists, import from csv
            else:
                complete_pair_tf.append([timeframe, pair, from_timestamp, collection_name, "is_new"])
    return complete_pair_tf

In [9]:
# Function to get the kline data from Binance
async def get_coinpair_kline(pair, timeframe, from_timestamp):
    client = await AsyncClient.create(BINANCE_API_KEY, BINANCE_API_SECRET)
    if timeframe == "1m":
        klines = await client.get_historical_klines(pair, Client.KLINE_INTERVAL_1MINUTE, from_timestamp)
    elif timeframe == "5m":
        klines = await client.get_historical_klines(pair, Client.KLINE_INTERVAL_5MINUTE, from_timestamp)
    elif timeframe == "30m":
        klines = await client.get_historical_klines(pair, Client.KLINE_INTERVAL_30MINUTE, from_timestamp)
    elif timeframe == "1h":
        klines = await client.get_historical_klines(pair, Client.KLINE_INTERVAL_1HOUR, from_timestamp)
    elif timeframe == "1d":
        klines = await client.get_historical_klines(pair, Client.KLINE_INTERVAL_1DAY, from_timestamp)
    else:
        return
    await client.close_connection()
    return klines

In [10]:
async def get_binance_data(complete_pair_tf):
    headers=["open_time", "open", "high", "low","close","volume","close_time","quote_asset_volume","numer_trades","taker_base_volume","taker_quote_volume","ignore"]
    for query_pair in complete_pair_tf:
        collection_name = query_pair[1]+"_"+query_pair[0]
        print(f"Processing {collection_name}..")
        kline_list = await get_coinpair_kline(query_pair[1], query_pair[0], query_pair[2])
        # If it is updating the db then delete the first item as it repeats
        if query_pair[4] == "is_update":
            kline_list.pop(0)
            print(f"Updating {len(kline_list)} items..")
        else:
            print(f"Adding {len(kline_list)} items..")
        if len(kline_list) > 0:
            kline_df = pd.DataFrame(kline_list, columns=headers)
            kline_df['open_time'] = kline_df['open_time'].values.astype(dtype='datetime64[ms]')
            kline_df['close_time'] = kline_df['close_time'].values.astype(dtype='datetime64[ms]')
            kline_df[["open", "high", "low","close","volume","quote_asset_volume","taker_base_volume","taker_quote_volume"]] = kline_df[["open", "high", "low","close","volume","quote_asset_volume","taker_base_volume","taker_quote_volume"]].astype(float)
            kline_df = kline_df.drop(columns=['ignore'])
            kline_dict = kline_df.to_dict("records")
            #print(kline_dict[0])
            db[query_pair[3]].insert_many(kline_dict)
    print(f"Done :)")


Processing ADABUSD_1m..
Updating 50 items..
Processing SOLBUSD_1m..
Updating 50 items..
Processing BTCBUSD_1m..
Updating 50 items..
Processing ETHBUSD_1m..
Updating 50 items..
Processing LUNABUSD_1m..
Updating 50 items..
Processing DOGEBUSD_1m..
Updating 50 items..
Processing MATICBUSD_1m..
Updating 50 items..
Processing MANABUSD_1m..
Updating 50 items..
Processing ADABUSD_5m..
Updating 10 items..
Processing SOLBUSD_5m..
Updating 10 items..
Processing BTCBUSD_5m..
Updating 10 items..
Processing ETHBUSD_5m..
Updating 10 items..
Processing LUNABUSD_5m..
Updating 10 items..
Processing DOGEBUSD_5m..
Updating 10 items..
Processing MATICBUSD_5m..
Updating 10 items..
Processing MANABUSD_5m..
Updating 10 items..
Processing ADABUSD_30m..
Updating 1 items..
Processing SOLBUSD_30m..
Updating 1 items..
Processing BTCBUSD_30m..
Updating 1 items..
Processing ETHBUSD_30m..
Updating 1 items..
Processing LUNABUSD_30m..
Updating 1 items..
Processing DOGEBUSD_30m..
Updating 1 items..
Processing MATICBUSD

In [14]:
# Gets the coinpairs to process from the database
def get_keywords(coinpair):
    # Get the coinpairs from the Database
    db_coinpairs = db["coinpairs"].find({"pair": coinpair, "exchange" : "binance"})

    # convert the dictionary objects to dataframe
    binance_coinpairs_df = pd.DataFrame(db_coinpairs)

    # see the magic
    coinpair_list = list(binance_coinpairs_df['keywords'][0])
    
    return coinpair_list

# Gets the main coin name to process from the database
def get_main_coin(coinpair):
    # Get the coin name from the Database
    db_coinpairs = db["coinpairs"].find({"pair": coinpair, "exchange" : "binance"})

    # convert the dictionary objects to dataframe
    binance_coinpairs_df = pd.DataFrame(db_coinpairs)

    # see the magic
    coinpair_list = binance_coinpairs_df['main'][0]
    
    return coinpair_list

# Gets the coinpairs to process from the database
def get_coinpairs():
    # Get the coinpairs from the Database
    db_coinpairs = db["coinpairs"].find({"exchange" : "binance"})

    # Convert the dictionary objects to dataframe
    binance_coinpairs_df = pd.DataFrame(db_coinpairs)

    # Convert dataframe to list
    coinpair_list = list(binance_coinpairs_df['pair'])
    return coinpair_list

# Save trends to Database
def update_trends_db(coin_name, trends_df):
    # Resets the index
    trends_df = trends_df.reset_index()
    
    # Checks if there no data to update then skips it
    if len(trends_df) > 0:
        # Update the trends for the Main Coin in the Database
        db["trends_"+coin_name].insert_many(trends_df.to_dict("records"))

# Processes the Google Trends and returns the dataframe with the most relevant column
def process_trends(df):
    # Select non partial data
    new_df = df.copy()
    new_df = new_df[new_df['isPartial'] != True]
    
    # Drop Columns that are not needed
    new_df = new_df.drop(columns=['isPartial'])
    
    return new_df

def get_google_trends(coinpair, keywords, coin_name):
    # Gets current time in UTC
    now_utc = datetime.now(timezone.utc)
    # Pull the last date from Database
    collection_name ="trends_"+coin_name
    latest = db[collection_name].find().sort('date', -1 ).limit(1)
    # Sets start and end Dates
    date_start = latest[0]["date"] + timedelta(hours=1)
    date_end = now_utc
    # Pulls data from Google Trends
    try:
        df = pytrend.get_historical_interest(keywords, year_start=date_start.year, month_start=date_start.month, day_start=date_start.day, hour_start=date_start.hour, year_end=date_end.year, month_end=date_end.month, day_end=date_end.day, hour_end=date_end.hour, cat=0, geo='', gprop='', sleep=60)
        trends_df = process_trends(df)
        update_trends_db(coin_name, trends_df)
    except:
        print(f"Error with Google Trends, waiting 3 seconds..")
        time.sleep(3)
            
# Function to initialize the databse with the coinpairs registered
def update_google_trends(db, coinpairs):
    for coinpair in coinpairs:
        coin_name = get_main_coin(coinpair)
        print(f"Updating trends for {coin_name}..")
        keywords = get_keywords(coinpair)
        get_google_trends(coinpair, keywords, coin_name)
            
    print(f"Done pulling trends.")

In [15]:
# Get pair info to update DB
complete_pair_tf = get_pair_data(binance_timeframes_list, coinpair_list)

# Update DB with the latest binance Data
await get_binance_data(complete_pair_tf)

Updating trends for ADA..
The request failed: Google returned a response with code 400.
Updating trends for SOL..
The request failed: Google returned a response with code 400.
Updating trends for BTC..
The request failed: Google returned a response with code 400.
Updating trends for ETH..
The request failed: Google returned a response with code 400.
Updating trends for LUNA..
The request failed: Google returned a response with code 400.
Updating trends for DOGE..
The request failed: Google returned a response with code 400.
Updating trends for MATIC..
The request failed: Google returned a response with code 400.
Updating trends for MANA..
The request failed: Google returned a response with code 400.
Done pulling trends.


In [16]:
update_google_trends(db, coinpair_list)